In [ ]:
import os
import shutil

# Define paths
source_dir = r"C:\Users\bhish\Downloads\FullAndFinalPlantVillage\Plant Village Dataset\Train"
destination_dir =  "PlantVillage_100_DATASETBHISHAMGAHLAUT"

# Folders for train, test, and validation
splits = ["train", "test", "valid"]
split_sizes = {"train": 200, "test": 50, "valid": 50}

# Create destination folders if not exist
for split in splits:
    os.makedirs(os.path.join(destination_dir, split), exist_ok=True)

# Process each category
for category in os.listdir(source_dir):
    category_path = os.path.join(source_dir, category)

    # Skip if not a directory
    if not os.path.isdir(category_path):
        continue

    # Get all images in category folder
    images = sorted(os.listdir(category_path))  # Sorting ensures order

    # Create category subfolders in train, test, and valid
    for split in splits:
        os.makedirs(os.path.join(destination_dir, split, category), exist_ok=True)

    # Copy images
    start_idx = 0
    for split in splits:
        num_images = split_sizes[split]
        split_dest = os.path.join(destination_dir, split, category)

        for img in images[start_idx:start_idx + num_images]:
            src = os.path.join(category_path, img)
            dest = os.path.join(split_dest, img)
            shutil.copy(src, dest)

        start_idx += num_images

print("✅ Dataset successfully split into train, test, and valid folders.")


✅ Dataset successfully split into train, test, and valid folders.


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from PIL import Image
import random

# ✅ Dataset Paths (Modify These Paths as Needed)
train_dir = r"C:\Users\bhish\OneDrive\Desktop\ML\NEWMODELS\PlantVillage_100_DATASETBHISHAMGAHLAUT\train"
val_dir = r"C:\Users\bhish\OneDrive\Desktop\ML\NEWMODELS\PlantVillage_100_DATASETBHISHAMGAHLAUT\valid"
test_dir = r"C:\Users\bhish\OneDrive\Desktop\ML\NEWMODELS\PlantVillage_100_DATASETBHISHAMGAHLAUT\test"

# ✅ Image Parameters
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32

# ✅ Data Augmentation & Normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1./255)  # Only Normalization
test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize test images

# ✅ Load Dataset
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=IMAGE_SIZE, batch_size=1, class_mode='categorical', shuffle=False)

# ✅ Get Class Labels
CLASS_NAMES = list(train_generator.class_indices.keys())

# ✅ Define CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Prevent Overfitting
    Dense(len(CLASS_NAMES), activation='softmax')  # Output layer for multiple classes
])

# ✅ Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ✅ Train Model
EPOCHS = 15
model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS)

# ✅ Save Model
model_path = r"C:\Users\bhish\OneDrive\Desktop\Agriculture_Project\Backend\WeatherAnalysis\my_model_ALL.keras"
model.save(model_path)
print(f"✅ Model saved at {model_path}")

# -----------------------------------------------
# 🚀 **Test Model on 20 Random Images from Test Set**
# -----------------------------------------------
# ✅ Get 20 Random Image Paths from Test Directory
test_images = []
for label in CLASS_NAMES:
    label_folder = os.path.join(test_dir, label)
    images = [os.path.join(label_folder, img) for img in os.listdir(label_folder) if img.endswith(('.jpg', '.png', '.jpeg'))]
    test_images.extend(images)

random.shuffle(test_images)  # Shuffle to pick random images
test_images = test_images[:20]  # Select 20 random test images

correct_predictions = 0
total_predictions = len(test_images)

# ✅ Function to Preprocess Image for Model
def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img = img.resize(IMAGE_SIZE)
    img_array = np.array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# ✅ Perform Predictions
for img_path in test_images:
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)

    predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
    confidence = round(100 * np.max(predictions[0]), 2)

    actual_class = img_path.split(os.path.sep)[-2]  # Get actual class from folder name

    if predicted_class == actual_class:
        correct_predictions += 1

    print(f"📸 Image: {os.path.basename(img_path)}")
    print(f"✅ Actual Class: {actual_class}")
    print(f"🔍 Predicted Class: {predicted_class} (Confidence: {confidence}%)")
    print("-" * 50)

# ✅ Calculate Accuracy
accuracy = (correct_predictions / total_predictions) * 100
print(f"🎯 Model Accuracy on 20 Test Images: {accuracy:.2f}% ({correct_predictions}/{total_predictions} Correct Predictions)")


Found 4172 images belonging to 18 classes.
Found 1141 images belonging to 18 classes.
Found 1140 images belonging to 18 classes.


C:\Users\bhish\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\bhish\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 237s 2s/step - accuracy: 0.1551 - loss: 2.8586 - val_accuracy: 0.4049 - val_loss: 1.9083
Epoch 2/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.3611 - loss: 1.9961 - val_accuracy: 0.5451 - val_loss: 1.4579
Epoch 3/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.4136 - loss: 1.7655 - val_accuracy: 0.6240 - val_loss: 1.2131
Epoch 4/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.4692 - loss: 1.5941 - val_accuracy: 0.6231 - val_loss: 1.1523
Epoch 5/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - accuracy: 0.4784 - loss: 1.5148 - val_accuracy: 0.6152 - val_loss: 1.2286
Epoch 6/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.5240 - loss: 1.3669 - val_accuracy: 0.5890 - val_loss: 1.2071
Epoch 7/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.5186 - loss: 1.3485 - val_accuracy: 0.6661 - val_loss: 0.9241
Epoch 8/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.5488 - loss: 1.2673 - val_accu